In [ ]:
import sys, os

root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

In [ ]:
from src.data.load_cifrar100 import *

train_loader, val_loader, test_loader = get_cifar100_dataloaders(
    batch_size=64,
    data_dir="./data",
    num_workers=2,
    val_split=0.1,
    img_size=32 , seed=7)

---


In [ ]:
from src.training.train_full_model import *
from stage_config import * 
from Model_A_OutGridNet import * 
from Model_B_OutGridNet import *

def cifar32_stages_t4_tinyplus(drop_path=0.08):
    # resoluciones: 64 -> 32 -> 16 -> 8
    return [
        StageCfg(dim=64,  depth=2, num_heads=2,  grid_size=8, outlook_heads=2,  drop_path=drop_path),
        StageCfg(dim=128, depth=2, num_heads=4,  grid_size=8, outlook_heads=4,  drop_path=drop_path),
        StageCfg(dim=256, depth=3, num_heads=8, grid_size=4, outlook_heads=8, drop_path=drop_path),
        StageCfg(dim=384, depth=1, num_heads=6,  grid_size=2, outlook_heads=6,  drop_path=drop_path),]

stages = cifar32_stages_t4_tinyplus()

model = MaxOutNet(
    num_classes=100,
    stages=stages,
    stem_dim=64,
    dpr_max=0.1)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [35]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n_params = count_trainable_parameters(model)
print(f"Trainable parameters: {n_params:,}")

Trainable parameters: 14,599,198


In [39]:
import random, numpy as np

seed = 77
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

random.seed(seed);
np.random.seed(seed)
torch.backends.cudnn.benchmark = True


history, model = train_model(
    model=model,
    train_loader=train_loader,
    epochs=100,
    val_loader=val_loader,
    device=device,

    lr=5e-4,
    weight_decay=0.05,

    autocast_dtype="fp16" if device == "cuda" else "fp32",
    use_amp=(device == "cuda"),
    grad_clip_norm=1.0,

    warmup_ratio=0.05,
    min_lr=1e-6,

    label_smoothing=0.1,

    print_every=400,
    save_path="best_maxout_medium.pt",
    last_path="last_maxout_medium.pt",
    resume_path=None,

    # Augmentations
    mix_prob=0.5,
    mixup_alpha=0.8,
    cutmix_alpha=1.0,

    num_classes=100,
    channels_last=True)


=== Run config ===
device=cuda | amp=True | autocast_dtype=fp16 | channels_last=True
epochs=100 | steps/epoch=704 | total_steps=70400 | warmup_steps=3520
batch_size=64 | input_shape=(64, 3, 32, 32) | num_classes=100
opt=AdamW | lr=0.0005 | wd=0.05 | grad_clip_norm=1.0
aug: mix_prob=0.5 | mixup_alpha=0.8 | cutmix_alpha=1.0 | label_smoothing=0.1

=== Epoch 1/100 ===
[train step 400/704] loss 3.6520 | top1 16.77% | top3 32.98% | top5 42.66% | 261.1 img/s | lr 5.68e-05 | gnorm 4.709 | clip 100.0% | oflow 0 | nonfinite 0 | scale 65536.0
[train step 704/704] loss 3.5662 | top1 18.60% | top3 35.44% | top5 45.14% | 261.8 img/s | lr 1.00e-04 | gnorm 5.064 | clip 100.0% | oflow 0 | nonfinite 0 | scale 65536.0
[Train] loss 3.5662 | top1 18.60% | top3 35.44% | top5 45.14% | lr 1.00e-04 | grad_norm 5.064 | clip 100.0% | amp_overflows 0 | nonfinite_loss 0 | scale 65536.0
[Train] mem_peak alloc 2.51 GiB | reserved 2.99 GiB
[Val]   loss 3.1531 | top1 23.68% | top3 42.60% | top5 53.10%
[Val]   mem_peak

In [89]:
free_all_cuda("optimizer", "scaler", "scheduler", "batch", "loss", "outputs", "logits")

[CUDA] allocated=790.9 MB | reserved(cache)=826.0 MB


---

In [ ]:
from src.training.eval_one_epoch_logs import *

avg_loss, metrics = evaluate_one_epoch_logs(
    model=model,
    dataloader=test_loader,
    device="cuda",
    use_amp=True,
    autocast_dtype="fp16",     
    channels_last=False,      
    measure_flops=True,       
    flops_warmup_batches=1)

print(f"loss: {avg_loss:.4f}")
print(f"top1: {metrics['top1']:.2f} | top3: {metrics['top3']:.2f} | top5: {metrics['top5']:.2f}")

print(f"throughput: {metrics['imgs_per_sec']:.1f} imgs/s | epoch: {metrics['epoch_time_sec']:.2f}s | ms/batch: {metrics['ms_per_batch']:.2f}")
print(f"GPU mem: alloc={metrics['gpu_mem_allocated_mib']:.0f} MiB | reserved={metrics['gpu_mem_reserved_mib']:.0f} MiB | peak={metrics['gpu_mem_peak_allocated_mib']:.0f} MiB")
print(f"model: params={int(metrics['model_params']):,} | param_size={metrics['model_param_size_mib']:.1f} MiB")

flops = metrics["flops_per_forward"]
macs  = metrics["macs_per_forward"]

print(
    f"Compute (per forward): "
    f"{format_ops(flops, 'FLOPs')} | "
    f"{format_ops(macs, 'MACs')}")

Unsupported operator aten::silu_ encountered 28 time(s)
Unsupported operator aten::softmax encountered 16 time(s)
Unsupported operator aten::im2col encountered 8 time(s)
Unsupported operator aten::mul encountered 56 time(s)
Unsupported operator aten::sum encountered 8 time(s)
Unsupported operator aten::add encountered 40 time(s)
Unsupported operator aten::gelu encountered 16 time(s)
Unsupported operator aten::sigmoid encountered 8 time(s)
Unsupported operator aten::mean encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
stages.0.1.dp2, stages.0.1.dp3, stages.0.1.outlook.dp1, stages.0.1.outlook.dp2, stages.1.0.dp2, stages.1.0.dp3, stages.1.0.outlook.dp1, stages.1.0.outlook.dp2, stages.1.1.dp2, stage

loss: 0.7299
top1: 80.85 | top3: 92.60 | top5: 95.51
throughput: 673.0 imgs/s | epoch: 14.86s | ms/batch: 88.77
GPU mem: alloc=791 MiB | reserved=1194 MiB | peak=1060 MiB
model: params=14,599,198 | param_size=55.7 MiB
Compute (per forward): 915.15 MFLOPs | 457.58 MMACs
